<a href="https://colab.research.google.com/github/KatBotkowska/WiT_workshops/blob/main/chocolate_bar_ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import pandas as pd
import numpy as np
import io
from unicodedata import normalize

In [2]:
#podłączenie google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Pobranie danych z Kaggle - nie odpalać

In [3]:
root_path = 'gdrive/My Drive/WiT_workshops/'

In [4]:
cd 'gdrive/My Drive/WiT_workshops/'

/content/gdrive/My Drive/WiT_workshops


In [5]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"kbotkowska","key":"90ebb11108c952959800f5549af3472d"}'}

In [6]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [7]:
!kaggle datasets download -d rtatman/chocolate-bar-ratings
# !kaggle datasets download -d sid321axn/beijing-multisite-airquality-data-set

chocolate-bar-ratings.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
!unzip -q chocolate-bar-ratings.zip -d [datasets]

replace [datasets]/flavors_of_cacao.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


#Czyszczenie danych

In [142]:
choco = pd.read_csv('/content/gdrive/MyDrive/WiT_workshops/[datasets]/flavors_of_cacao.csv', encoding='utf-8')

In [143]:
choco.head()

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru


In [144]:
choco.columns

Index(['Company \n(Maker-if known)', 'Specific Bean Origin\nor Bar Name',
       'REF', 'Review\nDate', 'Cocoa\nPercent', 'Company\nLocation', 'Rating',
       'Bean\nType', 'Broad Bean\nOrigin'],
      dtype='object')

In [145]:
#usunięcie znaku nowej linii \n z nazw kolumn
choco.columns = [column.replace('\n', ' ') for column in choco.columns.values]
choco.columns

Index(['Company  (Maker-if known)', 'Specific Bean Origin or Bar Name', 'REF',
       'Review Date', 'Cocoa Percent', 'Company Location', 'Rating',
       'Bean Type', 'Broad Bean Origin'],
      dtype='object')

In [146]:
#zmiana nazwy kolumny - z Kaggle: A value linked to when the review was entered in the database. Higher = more recent
choco.rename({'REF': 'Review data reference'}, axis=1, inplace=True)

In [147]:
choco.shape

(1795, 9)

In [148]:
choco.dtypes

Company  (Maker-if known)            object
Specific Bean Origin or Bar Name     object
Review data reference                 int64
Review Date                           int64
Cocoa Percent                        object
Company Location                     object
Rating                              float64
Bean Type                            object
Broad Bean Origin                    object
dtype: object

In [149]:
choco.describe()

,Review data reference,Review Date,Rating
count,1795.000000,1795.000000,1795.000000
mean,1035.904735,2012.325348,3.185933
std,552.886365,2.927210,0.478062
min,5.000000,2006.000000,1.000000
25%,576.000000,2010.000000,2.875000
50%,1069.000000,2013.000000,3.250000
75%,1502.000000,2015.000000,3.500000
max,1952.000000,2017.000000,5.000000


In [150]:
choco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Company  (Maker-if known)         1795 non-null   object 
 1   Specific Bean Origin or Bar Name  1795 non-null   object 
 2   Review data reference             1795 non-null   int64  
 3   Review Date                       1795 non-null   int64  
 4   Cocoa Percent                     1795 non-null   object 
 5   Company Location                  1795 non-null   object 
 6   Rating                            1795 non-null   float64
 7   Bean Type                         1794 non-null   object 
 8   Broad Bean Origin                 1794 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 126.3+ KB


In [151]:
 #ile pustych danych (nulli) mamy w tabeli?
choco.isnull().sum()

Company  (Maker-if known)           0
Specific Bean Origin or Bar Name    0
Review data reference               0
Review Date                         0
Cocoa Percent                       0
Company Location                    0
Rating                              0
Bean Type                           1
Broad Bean Origin                   1
dtype: int64

In [152]:
#usuwa wiersze z pustymi danymi - tutaj są tylko 2 takie wiersze
choco.dropna(inplace=True)

In [153]:
 #ile pustych danych (nulli) mamy w tabeli?
choco.isnull().sum()

Company  (Maker-if known)           0
Specific Bean Origin or Bar Name    0
Review data reference               0
Review Date                         0
Cocoa Percent                       0
Company Location                    0
Rating                              0
Bean Type                           0
Broad Bean Origin                   0
dtype: int64

In [154]:
#sortowanie danych
choco.sort_values(['Rating', 'Cocoa Percent'], ascending=False).head()

,Company (Maker-if known),Specific Bean Origin or Bar Name,Review data reference,Review Date,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
78,Amedei,Chuao,111,2007,70%,Italy,5.0,Trinitario,Venezuela
86,Amedei,Toscano Black,40,2006,70%,Italy,5.0,Blend,
1528,Soma,Crazy 88,1077,2013,88%,Canada,4.0,,"Guat., D.R., Peru, Mad., PNG"
1335,Pralus,Fortissima,93,2006,80%,France,4.0,"Criollo, Trinitario",Ecuador
1427,Scharffen Berger,San Juan de Cheni,959,2012,78%,U.S.A.,4.0,,Peru


In [155]:
choco.loc[1427]['Bean Type']

'\xa0'

In [157]:
funkcja1 = lambda x: x.replace(u'\xa0', u'')
choco['Bean Type'] = choco['Bean Type'].apply(funkcja1)

In [158]:
choco.loc[1427]['Bean Type']

''

In [159]:
choco.isnull().sum()

Company  (Maker-if known)           0
Specific Bean Origin or Bar Name    0
Review data reference               0
Review Date                         0
Cocoa Percent                       0
Company Location                    0
Rating                              0
Bean Type                           0
Broad Bean Origin                   0
dtype: int64

In [161]:
choco.isna().sum()

Company  (Maker-if known)           0
Specific Bean Origin or Bar Name    0
Review data reference               0
Review Date                         0
Cocoa Percent                       0
Company Location                    0
Rating                              0
Bean Type                           0
Broad Bean Origin                   0
dtype: int64

In [162]:
choco['Bean Type']

0                
1                
2                
3                
4                
          ...    
1790             
1791    Forastero
1792    Forastero
1793             
1794             
Name: Bean Type, Length: 1793, dtype: object